In [7]:
search_for = 72
start_from = 0
threads = 1
thread = 0
log_freq = 100000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.016999244689941406
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 525
all cases: 130050921
type: [1, 1, 1, 1, 72] 72
cases of this type: 26873856
100000 56.98076284150322
200000 51.160416375880345
300000 61.74806840998988
400000 51.687660120406306
500000 58.66801041095644
600000 46.17368014065836
700000 57.29143882700683
800000 44.83642556727601
900000 54.9666840105705
1000000 44.85714963116048
1100000 54.505074327679786
1200000 42.32686364195263
1300000 52.192088745884135
1400000 44.2117056593742
1500000 47.871812367642185
1600000 46.851773886325645
1700000 43.22183212970497
1800000 52.79449987059967
1900000 43.861726463876984
2000000 51.26078135180338
2100000 41.63643240657347
2200000 52.178905046696194
2300000 41.657806953685636
2400000 53.34712751931085
2500000 41.43526433100761
2600000 46.72470380098737
2700000 47.69861823418497
2800000 47.18833179313657
2900000 45.95324298882488
3000000 48.49349716977

29500000 20.625372429269227
29600000 22.99852008146938
29700000 16.122665806288424
29800000 20.416389507302657
29900000 22.190879818903298
30000000 21.20428978424929
30100000 14.854866681421408
30200000 19.7229182590815
30300000 23.579015907667525
30400000 21.079618649811454
30500000 14.675429976072271
30600000 20.146889321672205
30700000 22.17303116597213
30800000 18.01806719323689
30900000 15.22838990846677
31000000 22.120863598358234
31100000 22.0407102467573
31200000 15.433242082216307
31300000 19.248483162763222
31400000 22.065375291861034
31500000 21.858907380513312
31600000 14.165665525947487
31700000 19.91584710334001
31800000 21.16234734829949
31900000 20.419377700072083
32000000 12.826926110246287
32100000 21.78124750701932
32200000 21.73057638142917
32300000 16.630398421358446
32400000 16.929794794617404
32500000 21.574389135947758
32600000 21.555333620966234
32700000 14.108076397327858
32800000 21.615284521096772
32900000 24.90628816401562
33000000 24.067517033621055
331000

58600000 6.740158648486545
58700000 7.562818572644457
58800000 5.286786654166657
58900000 5.059576311838358
59000000 6.986998174301792
59100000 6.4311185261765065
59200000 4.71771985375812
59300000 5.328011940020596
59400000 6.271861754474512
59500000 5.762922545447872
59600000 4.03160809108526
59700000 5.781118464861322
59800000 6.054273468154442
59900000 5.248188612898763
60000000 3.8044253856502435
60100000 5.921854866204617
60200000 6.167786800096608
60300000 4.718171482347597
60400000 4.377209798161826
best so far: 0
type: [1, 1, 1, 8, 9] 72
cases of this type: 3359232
60500000 5.5491530247117336
60600000 5.9862347909713804
60700000 4.022023392095382
60800000 5.126728645947302
60900000 6.533053625317068
61000000 4.698589645584862
61100000 4.26396585305641
61200000 5.324665318965436
61300000 6.287062128836204
61400000 4.199134494214003
61500000 4.318692191292965
61600000 5.702894616370073
61700000 6.199063537251467
61800000 4.245055449122731
61900000 4.486690952492058
62000000 5.99

86000000 2.1020186281377056
86100000 2.6640298505072115
86200000 2.1169125853476074
86300000 2.479396949160171
86400000 2.186026850518676
86500000 2.3505369081084067
86600000 2.302446755199784
86700000 2.220079277983195
best so far: 0
type: [1, 1, 2, 6, 6] 72
cases of this type: 1119744
86800000 2.667673026166302
86900000 2.174129218434198
87000000 2.3428121742222467
87100000 2.1254990307053485
87200000 2.1954824165459508
87300000 1.9947253045394202
87400000 2.240341727452801
87500000 1.9806288345814618
87600000 2.305931229092038
87700000 1.944116811508702
87800000 2.226442089836313
best so far: 0
type: [1, 1, 2, 9, 4] 72
cases of this type: 746496
87900000 2.0206889358428297
88000000 2.35237565071791
88100000 2.474275212127139
88200000 2.2513387500519437
88300000 2.187489267281204
88400000 2.198397497357511
88500000 2.2161694621008037
88600000 2.123039766396196
best so far: 0
type: [1, 1, 2, 12, 3] 72
cases of this type: 559872
88700000 2.290433005513284
88800000 2.336715899287323
889

104500000 1.477445618929198
104600000 2.1532872166156944
104700000 1.990153305162817
104800000 1.998157819003287
104900000 1.9674135064027025
105000000 1.9564852873670757
105100000 1.9056718436161457
105200000 1.9207342143802304
105300000 1.9723120197491704
105400000 1.9420896922550586
105500000 1.9530550170495224
105600000 1.9152437245462537
105700000 1.907907858281531
105800000 1.8446609989846408
105900000 2.038490596278568
106000000 1.3341376480541567
106100000 0.7509761045775863
106200000 0.7488504062672954
106300000 0.7585053813069191
106400000 0.7410136486203471
106500000 1.7972228172748208
106600000 1.905205386389969
106700000 1.8023232304113368
106800000 1.8380397812874854
106900000 1.8432829932192227
107000000 1.8081707569289578
107100000 1.7574307920681973
107200000 1.7880586492924895
107300000 1.749941298326942
107400000 1.679861974469751
107500000 1.72074105202703
107600000 1.7600810584466797
107700000 1.765241668772767
107800000 1.862988424582612
best so far: 0
type: [1, 2

best so far: 0
type: [1, 3, 6, 2, 2] 72
cases of this type: 13824
best so far: 0
type: [1, 3, 6, 4, 1] 72
cases of this type: 6912
119700000 0.30981757965788836
best so far: 0
type: [1, 3, 8, 1, 3] 72
cases of this type: 15552
best so far: 0
type: [1, 3, 8, 3, 1] 72
cases of this type: 5184
best so far: 0
type: [1, 3, 12, 1, 2] 72
cases of this type: 6912
best so far: 0
type: [1, 3, 12, 2, 1] 72
cases of this type: 3456
best so far: 0
type: [1, 3, 24, 1, 1] 72
cases of this type: 1728
best so far: 0
type: [1, 4, 1, 1, 18] 72
cases of this type: 419904
119800000 0.5213110642324746
119900000 0.5887661540004955
120000000 0.3138758182330072
120100000 0.5576769780140447
best so far: 0
type: [1, 4, 1, 2, 9] 72
cases of this type: 209952
120200000 0.522342423864723
120300000 0.337177751524053
best so far: 0
type: [1, 4, 1, 3, 6] 72
cases of this type: 139968
120400000 0.39152310188493394
120500000 0.356959458881708
best so far: 0
type: [1, 4, 1, 6, 3] 72
cases of this type: 69984
best so far:

126600000 0.13049609521585503
best so far: 0
type: [2, 1, 2, 9, 2] 72
cases of this type: 46656
best so far: 0
type: [2, 1, 2, 18, 1] 72
cases of this type: 23328
best so far: 0
type: [2, 1, 3, 1, 12] 72
cases of this type: 186624
126700000 0.10917044351574978
126800000 0.13450146078499955
best so far: 0
type: [2, 1, 3, 2, 6] 72
cases of this type: 93312
126900000 0.13817385413627029
best so far: 0
type: [2, 1, 3, 3, 4] 72
cases of this type: 62208
127000000 0.10503365240859257
best so far: 0
type: [2, 1, 3, 4, 3] 72
cases of this type: 46656
best so far: 0
type: [2, 1, 3, 6, 2] 72
cases of this type: 31104
127100000 0.10697058293988439
best so far: 0
type: [2, 1, 3, 12, 1] 72
cases of this type: 15552
best so far: 0
type: [2, 1, 4, 1, 9] 72
cases of this type: 104976
127200000 0.09939864602859019
best so far: 0
type: [2, 1, 4, 3, 3] 72
cases of this type: 34992
best so far: 0
type: [2, 1, 4, 9, 1] 72
cases of this type: 11664
best so far: 0
type: [2, 1, 6, 1, 6] 72
cases of this type:

best so far: 0
type: [3, 1, 8, 1, 3] 72
cases of this type: 5184
best so far: 0
type: [3, 1, 8, 3, 1] 72
cases of this type: 1728
best so far: 0
type: [3, 1, 12, 1, 2] 72
cases of this type: 2304
best so far: 0
type: [3, 1, 12, 2, 1] 72
cases of this type: 1152
best so far: 0
type: [3, 1, 24, 1, 1] 72
cases of this type: 576
best so far: 0
type: [3, 2, 1, 1, 12] 72
cases of this type: 41472
best so far: 0
type: [3, 2, 1, 2, 6] 72
cases of this type: 20736
129500000 0.019594987489312224
best so far: 0
type: [3, 2, 1, 3, 4] 72
cases of this type: 13824
best so far: 0
type: [3, 2, 1, 4, 3] 72
cases of this type: 10368
best so far: 0
type: [3, 2, 1, 6, 2] 72
cases of this type: 6912
best so far: 0
type: [3, 2, 1, 12, 1] 72
cases of this type: 3456
best so far: 0
type: [3, 2, 2, 1, 6] 72
cases of this type: 10368
best so far: 0
type: [3, 2, 2, 2, 3] 72
cases of this type: 5184
best so far: 0
type: [3, 2, 2, 3, 2] 72
cases of this type: 3456
best so far: 0
type: [3, 2, 2, 6, 1] 72
cases of t

best so far: 0
type: [8, 1, 1, 3, 3] 72
cases of this type: 2187
best so far: 0
type: [8, 1, 1, 9, 1] 72
cases of this type: 729
best so far: 0
type: [8, 1, 3, 1, 3] 72
cases of this type: 729
best so far: 0
type: [8, 1, 3, 3, 1] 72
cases of this type: 243
best so far: 0
type: [8, 1, 9, 1, 1] 72
cases of this type: 81
best so far: 0
type: [8, 3, 1, 1, 3] 72
cases of this type: 243
best so far: 0
type: [8, 3, 1, 3, 1] 72
cases of this type: 81
best so far: 0
type: [8, 3, 3, 1, 1] 72
cases of this type: 27
best so far: 0
type: [8, 9, 1, 1, 1] 72
cases of this type: 9
best so far: 0
type: [9, 1, 1, 1, 8] 72
cases of this type: 4096
best so far: 0
type: [9, 1, 1, 2, 4] 72
cases of this type: 2048
best so far: 0
type: [9, 1, 1, 4, 2] 72
cases of this type: 1024
best so far: 0
type: [9, 1, 1, 8, 1] 72
cases of this type: 512
best so far: 0
type: [9, 1, 2, 1, 4] 72
cases of this type: 1024
best so far: 0
type: [9, 1, 2, 2, 2] 72
cases of this type: 512
best so far: 0
type: [9, 1, 2, 4, 1] 72
